In [ ]:
import os
import torch
import pandas as pd

import kaleido #required
kaleido.__version__ #0.2.1
import plotly
plotly.__version__ #5.5.0
#now this works:
import plotly.graph_objects as go
import plotly.express as px

import plotly.io as pio   
pio.kaleido.scope.mathjax = None

import numpy as np
import re
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.stats import median_abs_deviation
from sklearn.feature_selection import VarianceThreshold

from copairs import map
from copairs.matching import assign_reference_index

In [2]:
def load_metrics(dict_metrics):
    for key, value in dict_metrics.items():
        tensors = []
        filenames = sorted([f for f in os.listdir(value) if f.endswith('.pt')], key=lambda x: int(re.findall(r'\d+', x)[0]))
        for filename in filenames:
            file_path = os.path.join(value, filename)
            tensor = torch.load(file_path)
            tensors.append(tensor)
    
        dict_metrics[key] = torch.cat(tensors, dim=0)
    return dict_metrics

# Loading metadata

In [ ]:
path = "/folder1/folder2!cpg0000-jump-pilot/csv/"

list_plates_pt = sorted([f for f in os.listdir("/folder1/folder2/cpg0000-jump-pilot/metrics/real_openphenom") if f.endswith('.pt')], key=lambda x: int(re.findall(r'\d+', x)[0]))
csv_files = [x.split(".")[0] + ".csv" for x in list_plates_pt]

dfs = [pd.read_csv(path + file) for file in csv_files]
df = pd.concat(dfs, ignore_index=True)

df['row_letter'] = df['row'].apply(lambda x: chr(64 + x))
df['column_str'] = df['column'].apply(lambda x: f"{x:02d}")
df['well'] = df['row_letter'] + df['column_str']

df_platemap = pd.read_csv("cpg0000-jump-pilot/JUMP-Target-1_compound_platemap.txt", sep="	")
df_exp = pd.read_csv("cpg0000-jump-pilot/experiment-metadata.tsv", sep="	")
df_platemap['broad_sample'] = df_platemap['broad_sample'].fillna('DMSO')

df = pd.merge(df, df_platemap, left_on=['well'], right_on=['well_position'], how='inner')
df = pd.merge(df, df_exp, on="Assay_Plate_Barcode")

In [8]:
df_platemap

,well_position,broad_sample,solvent
0,A01,BRD-A86665761-001-01-1,DMSO
1,A02,DMSO,DMSO
2,A03,BRD-A22032524-074-09-9,DMSO
3,A04,BRD-A01078468-001-14-8,DMSO
4,A05,BRD-K48278478-001-01-2,DMSO
...,...,...,...
379,P20,BRD-K68982262-001-01-4,DMSO
380,P21,BRD-K24616672-003-20-1,DMSO
381,P22,BRD-A82396632-008-30-8,DMSO
382,P23,BRD-K61250553-003-30-6,DMSO


In [46]:
def averaged_by_well(features_fov, df_fov, agg_func=np.median):

    assert len(features_fov) == len(df_fov)
    
    grouped = df_fov.groupby(["well", "Assay_Plate_Barcode"])

    features = []
    metadata = []

    for name, group in grouped:
        idx = group.index
        features.append(agg_func(features_fov[idx], axis=0).flatten())
        metadata.append(group.iloc[0].drop("FOV"))

    return np.stack(features), pd.DataFrame(metadata).reset_index(drop=True)

def apply_pca(features_well, df_well):

    assert len(features_well) == len(df_well)
    
    dmso_indices = df_well[df_well["broad_sample"] == "DMSO"].index
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=256, whiten=True)),
        ('var_thresh', VarianceThreshold(threshold=1e-6))
    ])
    
    pipeline.fit(features_well[dmso_indices])
    return pipeline.transform(features_well)

def scale_by_plate(features_well, df_well, mean_func=np.median, std_func=median_abs_deviation):

    assert len(features_well) == len(df_well)
    
    for name, group in df_well.groupby("Assay_Plate_Barcode"):
        dmso_indices = group[group["broad_sample"] == "DMSO"].index
        plate_indices = group.index

        mean = mean_func(features_well[dmso_indices], axis=0)
        std = std_func(features_well[dmso_indices], axis=0)

        features_well[plate_indices] = (features_well[plate_indices] - mean) / std
        
    return features_well

In [47]:
def run_pipeline_activity(features_well, df_well):

    assert len(features_well) == len(df_well)
    
    reference_col = "Metadata_reference_index"
    
    df_activity = assign_reference_index(
        df_well,
        "broad_sample == 'DMSO'",  # condition to get reference profiles (neg controls)
        reference_col=reference_col,
        default_value=-1,
    )
    # positive pairs are replicates of the same treatment
    pos_sameby = ["broad_sample", reference_col]
    pos_diffby = []
    
    neg_sameby = []
    # negative pairs are replicates of different treatments
    neg_diffby = ["broad_sample", reference_col]
    df_activity["index"] = df_activity.index

    metadata = df_activity
    profiles = features_well
    
    activity_ap = map.average_precision(
        metadata, profiles, pos_sameby, pos_diffby, neg_sameby, neg_diffby, #distance="abs_cosine"
        distance="cosine"
    )
    activity_ap = activity_ap.query("broad_sample != 'DMSO'")  # remove DMSO
    
    activity_map = map.mean_average_precision(
        activity_ap, pos_sameby, null_size=1000000, threshold=0.05, seed=0
    )
    activity_map["-log10(p-value)"] = -activity_map["corrected_p_value"].apply(np.log10)

    return ((activity_map["corrected_p_value"] <= 0.05).sum() / len(activity_map))

In [ ]:
def loop_over_experiments(features_all, df_all, data_type, run_name):
    results = []

    for name, grouped in df_all.groupby(["Cell_type", "Time"]):
        experiment_indices = grouped.index
        features_fov = features_all[experiment_indices]
        df_fov = grouped.reset_index(drop=True)

        for agg_function in [np.mean, np.median]:
            for scaling_functions in [(np.mean, np.std)]:
                
                features_well, df_well = averaged_by_well(features_fov, df_fov, agg_function)
                features_well = apply_pca(features_well, df_well)
                features_well = scale_by_plate(features_well, df_well, scaling_functions[0], scaling_functions[1])
                
                fraction_retrieved = run_pipeline_activity(features_well, df_well)

                results.append({
                    "Run_name": run_name,
                    "Data_type": data_type,
                    "Cell_type": name[0],
                    "Time": name[1],
                    "Aggregation": agg_function.__name__,
                    "Scaling_center": scaling_functions[0].__name__,
                    "Scaling_scale": scaling_functions[1].__name__,
                    "Fraction_retrieved": fraction_retrieved
                })

    return pd.DataFrame(results)

In [ ]:
dict_metrics = {
    "real_openphenom_cpg": "/folder1/folder2/cpg0000-jump-pilot/metrics/real_openphenom",
    "real_openphenom_cpg_run2": "/folder1/folder2/cpg0000-jump-pilot-run2/metrics/real_openphenom",
    "real_openphenom_cpg_run3": "/folder1/folder2/cpg0000-jump-pilot-run3/metrics/real_openphenom",
}

dict_metrics = load_metrics(dict_metrics)

all_results = []

for run_name, metrics in dict_metrics.items():
    features = metrics.numpy().mean(1).reshape(-1, 5*384)
    df_result = loop_over_experiments(features, df, data_type="real-op", run_name=run_name)
    all_results.append(df_result)

final_df_real_op = pd.concat(all_results, ignore_index=True)

In [ ]:
dict_metrics = {
    "fake_openphenom_cpg": "/folder1/folder2/cpg0000-jump-pilot/metrics/fake_openphenom",
    "fake_openphenom_cpg_run2": "/folder1/folder2/cpg0000-jump-pilot-run2/metrics/fake_openphenom",
    "fake_openphenom_cpg_run3": "/folder1/folder2/cpg0000-jump-pilot-run3/metrics/fake_openphenom",
}

dict_metrics = load_metrics(dict_metrics)

all_results = []

for run_name, metrics in dict_metrics.items():
    features = metrics.numpy().mean(1).reshape(-1, 5*384)
    df_result = loop_over_experiments(features, df, data_type="fake-op", run_name=run_name)
    all_results.append(df_result)

final_df_fake_op = pd.concat(all_results, ignore_index=True)

In [ ]:
dict_metrics = {
    "real_inception_cpg": "/folder1/folder2/cpg0000-jump-pilot/metrics/real_inception",
    "real_inception_cpg_run2": "/folder1/folder2/cpg0000-jump-pilot-run2/metrics/real_inception",
    "real_inception_cpg_run3": "/folder1/folder2/cpg0000-jump-pilot-run3/metrics/real_inception",
}

dict_metrics = load_metrics(dict_metrics)

all_results = []

for run_name, metrics in dict_metrics.items():
    features = metrics.numpy().mean(1).reshape(-1, 2*2048)
    df_result = loop_over_experiments(features, df, data_type="real-inception", run_name=run_name)
    all_results.append(df_result)

final_df_real_inception = pd.concat(all_results, ignore_index=True)

In [ ]:
dict_metrics = {
    "fake_inception_cpg": "/folder1/folder2/cpg0000-jump-pilot/metrics/fake_inception",
    "fake_inception_cpg_run2": "/folder1/folder2/cpg0000-jump-pilot-run2/metrics/fake_inception",
    "fake_inception_cpg_run3": "/folder1/folder2/cpg0000-jump-pilot-run3/metrics/fake_inception",
}

dict_metrics = load_metrics(dict_metrics)

all_results = []

for run_name, metrics in dict_metrics.items():
    features = metrics.numpy().mean(1).reshape(-1, 2*2048)
    df_result = loop_over_experiments(features, df, data_type="fake-inception", run_name=run_name)
    all_results.append(df_result)

final_df_fake_inception = pd.concat(all_results, ignore_index=True)

In [ ]:
stacked_df_result = pd.concat([final_df_real_op, final_df_fake_op, final_df_real_inception, final_df_fake_inception], axis=0, ignore_index=True)

grouped_stats = (
    stacked_df_result
    .groupby(["Cell_type", "Time", "Aggregation", "Scaling_center", "Scaling_scale", "Data_type"])
    .agg(
        mean_fraction=('Fraction_retrieved', 'mean'),
        std_fraction=('Fraction_retrieved', 'std')
    )
    .reset_index()
)

In [79]:
grouped_stats[grouped_stats["Aggregation"] == "mean"].sort_values(["Cell_type","Time"])

,Cell_type,Time,Aggregation,Scaling_center,Scaling_scale,Data_type,mean_fraction,std_fraction
0,A549,24,mean,mean,std,fake-inception,0.846405,0.003268
1,A549,24,mean,mean,std,fake-op,0.821351,0.004992
2,A549,24,mean,mean,std,real-inception,0.846405,0.000000
3,A549,24,mean,mean,std,real-op,0.803922,0.000000
8,A549,48,mean,mean,std,fake-inception,0.915033,0.003268
9,A549,48,mean,mean,std,fake-op,0.915033,0.003268
10,A549,48,mean,mean,std,real-inception,0.934641,0.000000
11,A549,48,mean,mean,std,real-op,0.924837,0.000000
16,U2OS,24,mean,mean,std,fake-inception,0.851852,0.004992
17,U2OS,24,mean,mean,std,fake-op,0.826797,0.005660


In [81]:
grouped_stats[grouped_stats["Aggregation"] == "mean"]["std_fraction"].max()

0.005660296756761081

In [80]:
grouped_stats[grouped_stats["Aggregation"] == "median"].sort_values(["Cell_type","Time"])

,Cell_type,Time,Aggregation,Scaling_center,Scaling_scale,Data_type,mean_fraction,std_fraction
4,A549,24,median,mean,std,fake-inception,0.906318,0.001887
5,A549,24,median,mean,std,fake-op,0.728758,0.006536
6,A549,24,median,mean,std,real-inception,0.872549,0.000000
7,A549,24,median,mean,std,real-op,0.722222,0.000000
12,A549,48,median,mean,std,fake-inception,0.950980,0.000000
13,A549,48,median,mean,std,fake-op,0.879085,0.003268
14,A549,48,median,mean,std,real-inception,0.960784,0.000000
15,A549,48,median,mean,std,real-op,0.882353,0.000000
20,U2OS,24,median,mean,std,fake-inception,0.847495,0.004992
21,U2OS,24,median,mean,std,fake-op,0.835512,0.001887


In [82]:
grouped_stats[grouped_stats["Aggregation"] == "median"]["std_fraction"].max()

0.008224220517724125